In [3]:
!pip install simpletransformers

ERROR: datasets 1.7.0 has requirement tqdm<4.50.0,>=4.27, but you'll have tqdm 4.61.0 which is incompatible.
  Found existing installation: huggingface-hub 0.0.9
    Uninstalling huggingface-hub-0.0.9:
      Successfully uninstalled huggingface-hub-0.0.9


In [9]:
from simpletransformers.classification import ClassificationModel
#?ClassificationModel
model = ClassificationModel('bert', 'hfl/chinese-roberta-wwm-ext-large')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mod

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

In [10]:
!wget https://raw.githubusercontent.com/Moximixi/Emotional-analysis-of-curriculum-review/main/%E9%BB%84%E5%BF%A0%E8%B4%A2/newtest.csv

--2021-06-06 15:10:44--  https://raw.githubusercontent.com/Moximixi/Emotional-analysis-of-curriculum-review/main/%E9%BB%84%E5%BF%A0%E8%B4%A2/newtest.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 843461 (824K) [text/plain]
Saving to: ‘newtest.csv’

newtest.csv         100%[===================>] 823.69K  --.-KB/s    in 0.06s   

2021-06-06 15:10:45 (13.6 MB/s) - ‘newtest.csv’ saved [843461/843461]



In [11]:
import pandas as pd 
df = pd.read_csv("newtest.csv")
df=df.sample(frac=1.0)    #打乱数据集
df=df.reset_index(drop=True)
df[0:7500]


,comment,sentiment
0,无222222,0
1,给他测完入党,0
2,这个软件真心不错,1
3,学到了有用知识,1
4,我是来听课的，不是来看老师的，老师的镜头太多了，严重影响我看题了,0
...,...,...
7495,感觉挺好，通过交流能学会很多知识。,1
7496,感谢老师，平易近人，深入浅出，喜欢你,1
7497,老师引经据典，博学多才！！,1
7498,请各位老师给个痛快话，这课程有没有证书，怎么申请，什么时候给？上一轮的课还没结果呢？,0


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train, test = train_test_split(df, test_size=.2, random_state=2)
train, valid = train_test_split(train, test_size=.2, random_state=2)
train.head()
#划分训练测试和验证集

,comment,sentiment
2041,照着课本读也叫国家精品课？\n,0
5492,无语简直无语这个问题,0
7112,非常好的呢\n,1
2620,老师讲的很详尽,1
3585,我把课件全下载下来，有时间再细看看的，讲的非常好！谢谢！,1


In [14]:
model.train_model(train, args = {'overwrite_output_dir': True, 'learning_rate': 1e-5})

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/7241 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/906 [00:00<?, ?it/s]

(906, 0.2577768896866249)

In [15]:
result, model_output, wrong_predictions = model.eval_model(test)
result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2263 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/283 [00:00<?, ?it/s]

{'auprc': 0.9636075553200334,
 'auroc': 0.9724368894173006,
 'eval_loss': 0.2236963354887,
 'fn': 43,
 'fp': 82,
 'mcc': 0.8890059491253809,
 'tn': 957,
 'tp': 1181}

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
Matrix = np.array([[result['tp'],result['fn']],[result['fp'],result['tn']]])
property_macro_list = []
TP_sum = 0
property_sum = sum(sum(Matrix))
Matrix_SumByCol = Matrix.sum(axis = 0)
for index in range(len(Matrix)):
    property_macro_list.append(Matrix[index][index] / Matrix_SumByCol[index])
    TP_sum += Matrix[index][index]
    
print("宏精确率:",sum(property_macro_list)/len(Matrix))
print("微精确率:",TP_sum/property_sum)

宏精确率: 0.9460376088677751
微精确率: 0.9447635881573133
